In [ ]:
import numpy.random as rand
import numpy as np
import numpy.linalg as la
import timeit
import time
from numpy.linalg import inv
from tempfile import TemporaryFile
import scipy.stats as stats

In [ ]:
"""
Generate the probability mass function (PMF) of Ising Model given the weight matrix W and 0 mean random variables.

The PMF P(x) for a given input x is proportional to exp(Transpose(x) W x/2)

Input: 
Weight matrix - W

Output: 
Probability Distribution - prob
"""
def gen_prob(W):
    d = W.shape[0] 
    prob = np.zeros(2**d)
    for i in range(2**d):
        x = np.ones((d,1))*-1
        x[d - len(list(bin(i)[2:])):, 0] = np.array(list(bin(i)[2:])).astype(int)*2-1
        prob[i] = np.exp(np.matmul(np.transpose(x), np.matmul(W, x))*0.5)
    prob = prob/sum(prob)
    return prob

In [ ]:
"""
Generate the probability distribution of Ising Model given the weight matrix W and non-mean random variables.

The PMF P(x) for a given input x is proportional to exp(Transpose(x) W x/2 + Transpose(x) b)

Input: 
Weight matrix - W, 
Bias - b

Output: 
Probability Distribution - prob
"""
def gen_prob_b(W, b):
    d = W.shape[0] 
    prob = np.zeros(2**d)
    for i in range(2**d):
        x = np.ones((d,1))*-1
        x[d - len(list(bin(i)[2:])):, 0] = np.array(list(bin(i)[2:])).astype(int)*2-1
        prob[i] = np.exp(np.matmul(np.transpose(x), np.matmul(W, x))*0.5 + np.dot(np.transpose(x), b))
    prob = prob/sum(prob)
    return prob

In [ ]:
"""
Generate random samples given a probability mass function.

Inputs: 
probability mass function - prob, 
number of samples - num_samples

Output: 
samples (shape - (num_samples x d) where d is the number of nodes) - bin_samples
"""
def gen_samples(prob, num_samples):
    d = int(np.log2(len(prob)))
    samples = rand.choice(2**d, size = (num_samples,), replace = True, p = prob)
    bin_samples = np.ones((num_samples, d))*-1
    for i in range(len(samples)):
        bin_samples[i, d - len(list(bin(samples[i])[2:])):] = np.array(list(bin(samples[i])[2:])).astype(int)*2-1
    return bin_samples
    
    
        

In [ ]:
"""
Generate the Weight matrix for a binary tree structured Ising model.

Input: 
Number of nodes (of the form 2^l-1 where l is the number of levels) - num_nodes
Minimum weight - min_w
Maximum weight - max_w

Output:
Weight matrix - W
"""
def gen_binary_tree_W(params):
    num_nodes, min_w, max_w = params['d'], params['min_w'], params['max_w']
    W = np.zeros((num_nodes, num_nodes))
    for i in range((num_nodes-1)/2):
        W[i,2*i+1] = rand.uniform(min_w, max_w)
        W[2*i+1,i] = W[i,2*i+1]
        W[i,2*i+2] = rand.uniform(min_w, max_w)
        W[2*i+2,i] = W[i,2*i+2]
    return W
  

In [ ]:
"""
Generate the Weight matrix for a star structured Ising model with one internal node and the remaining leaf nodes.

Input: 
Number of nodes - num_nodes
Minimum weight - min_w
Maximum weight - max_w

Output:
Weight matrix - W
"""
def gen_star_W(params):
    num_nodes, min_w, max_w = params['d'], params['min_w'], params['max_w']
    W = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes-1):
      W[0, i+1] = rand.uniform(min_w, max_w)
      W[i+1, 0] = W[0,i+1]
    return W

In [ ]:
"""
Generate the Weight matrix for a chain structured Ising model.

Input: 
Number of nodes - num_nodes
Minimum weight - min_w
Maximum weight - max_w

Output:
Weight matrix - W
"""
def gen_chain_W(params):
    num_nodes, min_w, max_w = params['d'], params['min_w'], params['max_w']
    W = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes-1):
        W[i, i+1] = rand.uniform(min_w, max_w)
        W[i+1, i] = W[i, i+1]
    return W


In [ ]:
"""
Checks if a set of 4 nodes is non-star structured. If it is, it returns the separation of the nodes in 2 groups of 2 nodes each.

Input:
The empirical covariance matrix of all the nodes - emp_cov
The set of 4 nodes that are checked for non-star - nodes
The threshold for the categorization as a star/non-star from Table 1 in the paper - eps

Output:
If the set of 4 nodes is non-star or not - non_star
If they are, return the two pairs - fin_pair1, fin_pair2
If not return empty lists - [], []

"""
def is_non_star(emp_cov, nodes, eps):
#     print eps
    non_star = False
    fin_pair1 = [-1,-1]
    fin_pair2 = [-1,-1]
    for i in range(3):
        for j in range(i+1, 4):
            pair1 = [nodes[i], nodes[j]]
            pair2 = list(set(nodes) - set(pair1))
            if (emp_cov[pair1[0], pair2[0]]*emp_cov[pair1[1], pair2[1]])/\
                (emp_cov[pair1[0], pair2[1]]*emp_cov[pair1[1], pair2[0]]) > eps\
                and (emp_cov[pair1[0], pair2[0]]*emp_cov[pair1[1], pair2[1]])/\
                (emp_cov[pair1[0], pair1[1]]*emp_cov[pair2[0], pair2[1]])<eps:                
                non_star = True
                fin_pair1[:] = pair1[:]
                fin_pair2[:] = pair2[:]
    if non_star == True:
        return non_star, fin_pair1, fin_pair2
    else:
        return non_star, [], []
    

In [ ]:
"""
Given a subtree and a node - root, this function returns all the nodes in the equivalence class containting root in (subtree U root).

Inputs:
emp_cov - Empirical covariance matrix of all the nodes.
root - The node whose equivalence class we intend to find.
prox1 - Proximal Set 1 of all the nodes.
prox2 - Proximal Set 2 of all the nodes.
subtree - The subset of the nodes considered while finding the equivalence class.
eps - The threshold used while classifying a set of 4 nodes as star/non-star.

Output:
EC - The list of nodes in the same equivalence cluster as root when we consider the node only in (subtree U root).
"""
def find_EC(emp_cov, root, prox1, prox2, subtree, eps):
    EC = []
    if (len(subtree) <= 2):
        return subtree
    common_nodes_big = list(set(prox1[root])& set(subtree))
    
    for i in common_nodes_big:
        is_leaf = True
        k1 = i
        count = 0
        while k1==i or k1 == root:
            k1 = common_nodes_big[count]
            count +=1
        sig_i_k1 = emp_cov[i,k1]
        sig_root_k1 = emp_cov[root, k1]
        common_nodes = list(set(prox2[root])& set(prox2[i]) & set(prox2[k1]) & set(subtree))
        for k2 in common_nodes:
            sig_root_k2 = emp_cov[root, k2] 
            sig_i_k2 = emp_cov[i,k2]
            if min((sig_i_k1*sig_root_k2) / (sig_i_k2*sig_root_k1),\
                   (sig_i_k2*sig_root_k1) / (sig_i_k1*sig_root_k2))<eps:
                is_leaf = False
        if is_leaf:
            EC.append(i)       
    return EC

In [ ]:
"""
A subroutine used to merge nodes in the same subtree.  
"""
def merge_splits(split1, split2):
    len_common_r_e = len(split1)
    merged = [-1]*len_common_r_e
    for i in range(len_common_r_e):
        if split1[i] == 1 or split2[i] == 1:
            merged[i] = 1
    return merged      
  
"""
Split the nodes in the common proximal sets of the root node and the external node into multiple subtrees.

Input:
emp_cov - Empirical covariance matrix of all the nodes.
root - Internal node
ext - Leaf node
prox1 - Proximal Set 1 of all the nodes.
prox2 - Proximal Set 2 of all the nodes.
prohibited - The set of nodes assigned to other subtrees in the previous iterations that should not be considered while splitting the current subtree.
eps - The threshold used while classifying a set of 4 nodes as star/non-star.

Output:
subtrees - A list of subtrees

"""  
def split_subtree(emp_cov, root, ext, prox1, prox2, prohibited, eps):
    common_root_ext = list(set(prox1[root]) & set(prox1[ext]) )
    len_common_r_e = len(common_root_ext)
    split = [[-1]*len_common_r_e for i in range(len_common_r_e)]
    for i in range(len_common_r_e):
        split[i][i] = 1
    
    for i in range(len_common_r_e):
        node_i = common_root_ext[i]
        if node_i in prohibited: 
            split[i][i] = -1
            continue      
        for j in range(len_common_r_e):
            node_j = common_root_ext[j]
            if node_j == node_i or node_j not in prox2[node_i]:
                continue
            nodes = [root, ext, node_i, node_j]
            status, pair1, pair2 = is_non_star(emp_cov, nodes, eps)
            if status and root in pair2 and ext in pair2:
                if node_j in prohibited:
                    split[i][:] = [-1]*len_common_r_e
                    break
                split[i][j] = 1

    deleted = []
    for i in range(len_common_r_e):
        first = 1
        for j in range(len_common_r_e):
            if j in deleted:
                continue
            if split[j][i] == 1:
                if first == 1:
                    first = 0
                    ind = j
                else:
                    split[ind] = merge_splits(split[ind], split[j])
                    deleted.append(j)   
    subtrees = []
    for j in range(len_common_r_e):
        if j in deleted:
            continue
        subtree_j = []
        for k in range(len_common_r_e):
            if split[j][k] == 1:               
                subtree_j.append(common_root_ext[k])
        subtrees.append(subtree_j)
    
    return subtrees
"""
The initialization of the algorithm where it finds a node with atleast one more node in its equivalence class.

Input:
emp_cov - Empirical covariance matrix of all the nodes.
prox1 - Proximal Set 1 of all the nodes.
prox2 - Proximal Set 2 of all the nodes.
eps - The threshold used while classifying a set of 4 nodes as star/non-star.
edges - The list of edges discovered by the algorithm. This is updated when the first equivalence cluster with more than one nodes is discovered.
prohibited - The set of nodes in the discovered equivalence cluster is added to the prohibited set so that they are not assigned to subtrees in the future.

Output:
i - A node with more than one nodes in its equivalence cluster.
EC_init - The list of all the nodes in the equivalence cluster containing i.
edges - The updated list of edges.
"""
  
def alg_init(emp_cov, prox1, prox2, eps, edges, prohibited):
    n = emp_cov.shape[0]
    subtree = range(n)
    for i in range(n):
        EC_init = find_EC(emp_cov, i, prox1, prox2, subtree, eps)
        len_EC_init = len(EC_init)
        if len_EC_init > 0:
            prohibited.append(i)
            for j in EC_init:
                edges.append([i,j])
                prohibited.append(j)
            break
    return i, EC_init, edges
  
"""
The recursive step of the algorithm. It updates the edges.

Input:
emp_cov - Empirical covariance matrix of all the nodes.
prox1 - Proximal Set 1 of all the nodes.
prox2 - Proximal Set 2 of all the nodes.
eps - The threshold used while classifying a set of 4 nodes as star/non-star.
edges - The list of edges discovered by the algorithm. 
xi - The node xext has an edge with.
xext - The leaf node.
prohibited - The set of nodes assigned previously to other subtrees.
"""  
def full_alg_recurse(emp_cov, prox1, prox2, eps, edges, xi, xext, prohibited):
    
    sub_subtrees = split_subtree(emp_cov, xext, xi, prox1, prox2, prohibited, eps)
    for i in sub_subtrees:
        for j in i:
            prohibited.append(j)
    num_sub_subtrees = len(sub_subtrees)  
    for i in range(num_sub_subtrees):
        local_prohibit = [-1]*len(prohibited)
        local_prohibit[:] = prohibited[:]
        for j in sub_subtrees[i]:
            local_prohibit.remove(j)
        EC = find_EC(emp_cov, xi, prox1, prox2, sub_subtrees[i], eps)
        len_EC = len(EC)
        
        for j in range(1, len_EC):
            edges.append([EC[j], EC[0]])
            local_prohibit.append(EC[j])
        if len_EC>0:
            edges.append([xi, EC[0]])
            local_prohibit.append(EC[0])
            full_alg_recurse(emp_cov, prox1, prox2, eps, edges, EC[0], xi, local_prohibit)

            
"""
Generates the two prox sets.
"""
def get_prox(emp_corr, thres1, thres2):
    prox1 = []
    prox2 = []
    n = emp_corr.shape[0]
    for i in range(n):
        prox1_i = []
        prox2_i = []
        for j in range(n):
            if j == i:
                continue
            if abs(emp_corr[i,j])>thres1:
                prox1_i.append(j)
            if abs(emp_corr[i,j])>thres2:
                prox2_i.append(j)
        prox1.append(prox1_i)
        prox2.append(prox2_i)
    return prox1, prox2
  
  
"""
This is the complete algorithm. Returns the edges learnt by the algorithm. If the algorithm fails at this stage, it returns an error.

Inputs:
sample_sig - Sample covariance matrix.
prox1 - Proximal Set 1 of all the nodes.
prox2 - Proximal Set 2 of all the nodes.
rho_max - Maximum correlation between 2 consecutive nodes.

Output:
edges - Edges learnt by the algorithm.
error - Returns an error if the algorithm fails

"""  
def find_tree(sample_sig, prox1, prox2, rho_max):
    n = sample_sig.shape[0]
    edges = []
    eps = (1+rho_max**2)/2
    prohibited = []
    root, EC_init, edges = alg_init(sample_sig, prox1, prox2, eps, edges, prohibited)
    error = 0
    if len(EC_init) == 0:
        return [], -1
    subtree = range(n)
    subtree = list(set(subtree) - set(EC_init))
    full_alg_recurse(sample_sig, prox1, prox2, eps, edges, root, EC_init[0], prohibited)
    return edges, error
  
"""
This removes any redundant edges.
"""
def deduplicate_edges(edges):
    for i in edges:
        for j in edges:
            if i==j:
                continue
            if set(i) == set(j):
                edges.remove(j)
    return edges

"""
Constructs all the trees in the equivalence class if the tree is chain-structured.
"""
def chain_edge_sets(params):
    d = params['d']
    edges_1 = []
    for i in range(d-1):
        edges_1.append(frozenset([i,i+1]))
    edges_1_set = set(edges_1)
    edges_1[1] = frozenset([0,2])
    edges_2_set = set(edges_1)
    edges_1[-2] = frozenset([d-3, d-1])
    edges_3_set = set(edges_1)
    edges_1[1] = frozenset([1,2])
    edges_4_set = set(edges_1)
    return [edges_1_set, edges_2_set, edges_3_set, edges_4_set]

In [ ]:
"""
Constructs all the trees in the equivalence class if the tree is star-structured.
"""
def star_edge_sets(params):
    d = params['d']
    edge_set_list = []
    for i in range(d):
        edge_list = []
        for j in range(d):
            if j == i:
                continue
            edge_list.append(frozenset([i,j]))
        edge_set = set(edge_list)
        edge_set_list.append(edge_set)
          
    return edge_set_list

def edges_as_set_of_sets(edges):
    edges_set = set([frozenset(i) for i in edges])
    return edges_set

In [ ]:
def corr(W, b1, b2):
    mean1 = (np.exp(b1)*(np.exp(W+b2)+np.exp(-W-b2)) - np.exp(-b1)*(np.exp(-W+b2)+np.exp(W-b2)))\
                /(np.exp(b1)*(np.exp(W+b2)+np.exp(-W-b2)) + np.exp(-b1)*(np.exp(-W+b2)+np.exp(W-b2)))
    mean2 = (np.exp(b2)*(np.exp(W+b1)+np.exp(-W-b1)) - np.exp(-b2)*(np.exp(-W+b1)+np.exp(W-b1)))\
                /(np.exp(b2)*(np.exp(W+b1)+np.exp(-W-b1)) + np.exp(-b2)*(np.exp(-W+b1)+np.exp(W-b1)))
    E_X1_X2 = (np.exp(W)*(np.exp(b1+b2)+np.exp(-b1-b2)) - np.exp(-W)*(np.exp(-b2+b1)+np.exp(b2-b1)))\
              /(np.exp(W)*(np.exp(b1+b2)+np.exp(-b1-b2)) + np.exp(-W)*(np.exp(-b2+b1)+np.exp(b2-b1)))
    corr = (E_X1_X2 - mean1*mean2)/np.sqrt((1-mean1**2)*(1-mean2**2))
    return corr

In [ ]:
# Initializing parameters and variables

def init_params():
    params = {}                                         # Dictionary of Parameters
    params['shape'] = 'star'                            # graph structure: 'star', 'chain'
    params['b_list'] = [0.0, 0.2, 0.4]                  # list of bias values
    params['num_samples_list'] = range(400, 3001, 1000) # list of number of samples
    params['d'] = 11                                    # number of nodes
    params['min_w'] = 0.7                               # minimum edge weight in the graph
    params['max_w'] = 1.2                               # maximum edge weight in the graph
    params['num_iter'] = 50                             # number of runs
    params['q_min'] = 0                                 # minimum probability of corruption for a node
    params['q_max'] = 0.1                               # maximum probability of corruption for a node
    
    results = {}                                        # Dictionary of results
    results['num_error_list'] = []                      # List of list for number of errors 
                                                        # size = len(params['b_list']) x len(num_samples_list)
    
    return params, results

# Defining main function
def main():
    params, results = init_params()
    for mod_b in params['b_list']:
        time_start = time.time()
        
        rand.seed(1)
        b = rand.uniform(mod_b, mod_b, params['d'])
        q = rand.uniform(params['q_min'], params['q_max'], params['d'])
        
        # Generating the weight matrix, Here we only consider two graph structures: star and chain
        # The code for other potential graph structures can be placed here
        # It should have two components: 
        # i) a function to generate the weight matrix W
        # ii) a function to find the equivalence class for that graph structure
        if params['shape'] == 'chain':
            W = gen_chain_W(params)
            equivalence_class = chain_edge_sets(params)
        elif params['shape'] == 'star':
            W = gen_star_W(params)
            equivalence_class = star_edge_sets(params)
        
        # Generate the probability distribution of Ising Model given the weight matrix W 
        # and non-mean random variables.
        prob = gen_prob_b(W, b)
        
        results['num_error_list'].append([])
        
        for num_samples in params['num_samples_list']:
            
            num_error = 0

            for itr in range(params['num_iter']):
                # Generate samples from the given distribution
                samples = gen_samples(prob, num_samples)
                noise = np.empty([num_samples, params['d']], dtype = int)
                
                for i in range(params['d']):
                    noise[:, i] = stats.bernoulli.rvs(1 - q[i], size = num_samples) * 2 - 1
                    
                # Generate noisy samples from clean samples
                noisy_samples = np.multiply(noise, samples)
                
                # Statistics of the noisy samples
                sample_mu =  np.array(1.0 / num_samples*np.sum(noisy_samples, axis=0)).reshape((1, params['d']))
                mu_max = np.max(sample_mu)
                emp_corr = 1.0 / (num_samples-1)*(np.matmul(np.transpose(noisy_samples), noisy_samples) - np.dot(np.transpose(sample_mu), sample_mu))
                corr_min = abs(corr(params['min_w'], mod_b, mod_b))
                corr_max = abs(corr(params['max_w'], 0, 0))
                
                # Define the algorithm parameters
                thres1 = (corr_min)**4 * (1 - 2 * params['q_max']) ** 2*(1-mu_max**2) / 2.0
                thres2 = min([thres1 * (1 - 2 * params['q_max']) * np.sqrt((1-mu_max**2)) / corr_max, thres1])
                
                # Run the algorithm subroutines: 
                # i) finding the proximal set 
                # ii) finding the equivalence tree
                prox1, prox2 = get_prox(emp_corr, thres1, thres2)
                edges, error = find_tree(emp_corr, prox1, prox2, corr_max)
                
                # Verify if the edges lie in the equivalence class
                edges_set = edges_as_set_of_sets(edges)
                
                # Calculate if the edge set was recovered correctly upto the equivalence class
                if edges_set not in equivalence_class or error ==-1:
                    num_error += 1
            
            # Store the results for a given bias value for all the samples in num_samples_list
            results['num_error_list'][len(results['num_error_list'])-1].append(num_error)
        
        print("Time Elapsed", time.time() - time_start, params['d'], num_samples, results['num_error_list'])
    
    return results
            


In [ ]:
# Here is where you run the code. 
results = main()